# How to build FFNN (f-lyr + p-lyr)

## Prepare environment and  define parameters 

In [ ]:
is_colab = True
import sys
if is_colab:
    !pip install -q torch==1.9.0
    !pip install -q torchvision==0.10.0
    !pip install -q qiskit==0.20.0
    !pip install qfnn
    !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1w9VRv0iVfsH20Kb_MkF3yFhFeiYDVy5n' -O model.tar.gz
    !tar zxvf /content/model.tar.gz

import torch
from qiskit import  QuantumCircuit
import functools
import qfnn
from qfnn.qf_fb.q_output import fire_ibmq,analyze,add_measure
from qfnn.qf_circ.f_lyr_circ import F_LYR_Circ
from qfnn.qf_circ.p_lyr_circ import P_LYR_Circ
from qfnn.qf_net.utils import binarize
from qfnn.qf_fb.q_input import UMatrixCircuit
from qfnn.qf_fb.c_input import load_data,to_quantum_matrix

print = functools.partial(print, flush=True)

################ Zhirui on 12-30-2020 ################
# path 
# remember to change the path on your computer
######################################################
if is_colab:
    data_path = '/content/data' #mnist  path
    resume_path = '/content/model/u2_p2/model_best.tar' #model path
else:
    data_path = 'your local path' #mnist  path
    resume_path = 'your local path' #model path


In [ ]:

################ Zhirui on 12-30-2020 ################
# Parameters of settings
######################################################
interest_num = [3,6]
img_size = 4
batch_size = 1# how many samples per batch to load
inference_batch_size = 1
isppd = False #is prepared data


## Load data

In [ ]:
train_loader, test_loader = load_data(interest_num,data_path,isppd,img_size,batch_size,inference_batch_size,False)


for batch_idx, (data, target) in enumerate(test_loader):
    torch.set_printoptions(threshold=sys.maxsize)
    print("Batch Id: {}, Target: {}".format(batch_idx,target))
    quantum_matrix = to_quantum_matrix(data)
    break

## Inference

In [ ]:
################ hzr on 12-30-2020 ################
# Get the parameters of the trained model
######################################################

# Model initialization
checkpoint = torch.load(resume_path, map_location='cpu')
print(checkpoint['state_dict']['fc0.weight'])
print(checkpoint['state_dict']['fc1.weight'])
weight_1 = checkpoint['state_dict']['fc0.weight']
weight_2 = checkpoint['state_dict']['fc1.weight']


### data encoding

In [ ]:
################ hzr on 12-30-2020 ################
# Generate the circuit of u-Matrix()
######################################################

#init circuit
circuit = QuantumCircuit()
#define your input and repeat number
u_mat = UMatrixCircuit(4,4)
#add input qubit to your circuit if needed
inputs = u_mat.add_input_qubits(circuit)
#add u-matrix to your circuit
u_mat.forward(circuit,inputs,quantum_matrix)

### the first layer

In [ ]:
################ hzr on 12-30-2020 ################
# Generate the circuit of f-layer
######################################################

#define your input and repeat number
f_layer = F_LYR_Circ(4,2)

#add qubits to your circuit if needed
aux = f_layer.add_aux(circuit)
f_layer_out_qubits = f_layer.add_out_qubits(circuit)

#add f-layer to your circuit
f_layer.forward(circuit,binarize(weight_1),inputs,f_layer_out_qubits,None,aux)

circuit.barrier()

circuit.draw('text',fold=300)

### the second layer

In [ ]:
################ Weiwen on 12-30-2020 ################
# Generate the circuit of p-layer
######################################################

#define your input and repeat number
p_layer = P_LYR_Circ(2,2)

#add output qubit to your circuit if needed
p_layer_output = p_layer.add_out_qubits(circuit)

#add p-layer to your circuit
p_layer.forward(circuit,binarize(weight_2),f_layer_out_qubits,p_layer_output)

#add measurement to your circuit if needed
add_measure(circuit,p_layer_output,'reg')

circuit.draw('text',fold=300)



### simulation



In [ ]:
################ Weiwen on 12-30-2020 ################
# Quantum simulation
######################################################


qc_shots=8192
opt_counts = fire_ibmq(circuit,qc_shots,True)
(opt_mycount,bits) = analyze(opt_counts)
opt_class_prob=[]
for b in range(bits):
    opt_class_prob.append(float(opt_mycount[b])/qc_shots)


print("Simulation Result :",opt_class_prob)
print("Prediction class: {}".format(opt_class_prob.index(max(opt_class_prob))))
print("Target class: {}".format(target[0]))
if opt_class_prob.index(max(opt_class_prob))==target[0]:
    print("Correct prediction")
else:
    print("Incorrect prediction")
print("="*30)

### classical inference

In [ ]:

from qfnn.qf_fb.c_qf_mixer import Net
from qfnn.qf_fb.c_input import ToQuantumData
neural_in_layers = 'u:2,p:2'
layers = []
for item1 in neural_in_layers.split(","):
    x= item1.split(":")
    layer =[]
    layer.append(x[0].strip())
    layer.append(int(x[1].strip()))
    layers.append(layer)

print("layers:",layers)
model = Net(img_size,layers,False,False)
model.load_state_dict(checkpoint["state_dict"])

to_quantum_data = ToQuantumData(img_size)
output_data = to_quantum_data(data)
output = model.forward(output_data,False)
print("Simulation Result:",output)